# 라이브러리 및 데이터 불러오기

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [43]:
base_path = "C:/Users/gmlkd/data/시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/"

train_err = pd.read_csv(base_path+"train_err_data.csv")
train_problem = pd.read_csv(base_path+"train_problem_data.csv")
train_quality = pd.read_csv(base_path+"train_quality_data.csv")
test_err = pd.read_csv(base_path+"test_err_data.csv")
test_quality = pd.read_csv(base_path+"test_quality_data.csv")
submission = pd.read_csv(base_path+"sample_submission.csv")

C:\Users\gmlkd\AppData\Local\Temp\ipykernel_20968\109563190.py:5: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  train_quality = pd.read_csv(base_path+"train_quality_data.csv")
C:\Users\gmlkd\AppData\Local\Temp\ipykernel_20968\109563190.py:7: DtypeWarning: Columns (4,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  test_quality = pd.read_csv(base_path+"test_quality_data.csv")


In [44]:
display(train_err.head(2))
display(test_err.head(2))
display(train_quality.head(2))
display(test_quality.head(2))
display(train_problem.head(2))

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1


,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
1,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0


,user_id,time
0,19224,20201102200000
1,23664,20201116140000


# 데이터 전처리

## 결측치 처리

In [45]:
# time column을 datetype으로 변경

train_err['time'] = pd.to_datetime(train_err['time'], format='%Y%m%d%H%M%S')
train_problem['time'] = pd.to_datetime(train_problem['time'], format='%Y%m%d%H%M%S')
train_quality['time'] = pd.to_datetime(train_quality['time'], format='%Y%m%d%H%M%S')
test_err['time'] = pd.to_datetime(test_err['time'], format='%Y%m%d%H%M%S')
test_quality['time'] = pd.to_datetime(test_quality['time'], format='%Y%m%d%H%M%S')

### train_err

In [46]:
train_err.isnull().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     1
dtype: int64

In [47]:
# errcode 40013일 때와 user_id, time, model_nm, fwver, errtype모두 같다
train_err.iloc[3825742:3825747]

,user_id,time,model_nm,fwver,errtype,errcode
3825742,13639,2020-11-21 17:40:58,model_2,04.33.1261,31,1
3825743,13639,2020-11-21 17:41:10,model_2,04.33.1261,31,0
3825744,13639,2020-11-21 19:17:18,model_2,04.33.1261,5,NaN
3825745,13639,2020-11-21 19:17:18,model_2,04.33.1261,5,40013
3825746,13639,2020-11-21 22:09:19,model_2,04.33.1261,15,1


In [48]:
# 따라서 같은 errcode일 것임을 유추하여 40013값을 넣어준다.
train_err.errcode = train_err.errcode.fillna('40013')

In [49]:
# model_nm, fwver, errcode encoding 필요

train_err.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16554663 entries, 0 to 16554662
Data columns (total 6 columns):
 #   Column    Dtype         
---  ------    -----         
 0   user_id   int64         
 1   time      datetime64[ns]
 2   model_nm  object        
 3   fwver     object        
 4   errtype   int64         
 5   errcode   object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 757.8+ MB


### train_quality

In [50]:
train_quality.isnull().sum()

time               0
user_id            0
fwver          40080
quality_0     144432
quality_1          0
quality_2      40113
quality_3          0
quality_4          0
quality_5         20
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

In [51]:
# 1. fwver 결측치

# case1. fwver=non을 이상치로 판단하고 모두 지우기
# case2. train_err에서 해당 user_id의 fwver값을 가져와 채우기

In [52]:
# train_err에는 fwver가 업그레이드 된 경우가 많이 있다.
# 결측치 비율이 적고, 어떤 fwver에서 로그가 찍혔는지 알기 힘드므로 case1 선택

fwver_null_index = train_quality[train_quality.fwver.isnull()].index

train_quality = train_quality.drop(fwver_null_index, axis=0)
train_quality.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 788544 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        788544 non-null  datetime64[ns]
 1   user_id     788544 non-null  int64         
 2   fwver       788544 non-null  object        
 3   quality_0   684192 non-null  float64       
 4   quality_1   788544 non-null  int64         
 5   quality_2   788511 non-null  float64       
 6   quality_3   788544 non-null  int64         
 7   quality_4   788544 non-null  int64         
 8   quality_5   788524 non-null  object        
 9   quality_6   788544 non-null  int64         
 10  quality_7   788544 non-null  object        
 11  quality_8   788544 non-null  object        
 12  quality_9   788544 non-null  object        
 13  quality_10  788544 non-null  object        
 14  quality_11  788544 non-null  int64         
 15  quality_12  788544 non-null  int64         
dtypes:

In [53]:
# 2. quality 결측치

# case1. 최빈값 채우기
# case2. 평균값 채우기

train_quality[['quality_0', 'quality_2']].describe()

,quality_0,quality_2
count,684192.000000,788511.000000
mean,4.148701,4.751094
std,479.315029,586.252469
min,-1.000000,-1.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,157667.000000,191859.000000


In [54]:
# -1, 0 ,1에 데이터가 편향되어있고, max값과 차이가 많이 난다.
# 따라서 case1. 최빈값으로 채우기 선택

for i in train_quality.columns[3:]:
    train_quality[i] = train_quality[i].fillna(train_quality[i].mode()[0])

In [55]:
train_quality.isnull().sum()

time          0
user_id       0
fwver         0
quality_0     0
quality_1     0
quality_2     0
quality_3     0
quality_4     0
quality_5     0
quality_6     0
quality_7     0
quality_8     0
quality_9     0
quality_10    0
quality_11    0
quality_12    0
dtype: int64

### test_err

In [56]:
test_err.isnull().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     4
dtype: int64

In [57]:
display(test_err.iloc[937965:937970])
display(test_err.iloc[4038890:4038895])
display(test_err.iloc[9486879:9486883])
display(test_err.iloc[10425470:10425475])

,user_id,time,model_nm,fwver,errtype,errcode
937965,30820,2020-11-15 03:59:00,model_2,04.33.1261,40,1
937966,30820,2020-11-15 03:59:02,model_2,04.33.1261,40,0
937967,30820,2020-11-15 04:43:17,model_2,04.33.1261,5,NaN
937968,30820,2020-11-15 04:43:17,model_2,04.33.1261,5,40053
937969,30820,2020-11-15 09:10:24,model_2,04.33.1261,15,1


,user_id,time,model_nm,fwver,errtype,errcode
4038890,33681,2020-11-03 11:01:47,model_2,04.33.1185,14,14
4038891,33681,2020-11-03 11:02:59,model_2,04.33.1185,7,14
4038892,33681,2020-11-03 11:02:59,model_2,04.33.1185,5,NaN
4038893,33681,2020-11-03 11:02:59,model_2,04.33.1185,5,40053
4038894,33681,2020-11-03 11:03:00,model_2,04.33.1185,6,14


,user_id,time,model_nm,fwver,errtype,errcode
9486879,38991,2020-11-27 18:59:52,model_2,04.33.1261,5,B-A8002
9486880,38991,2020-11-27 21:38:38,model_2,04.33.1261,5,40053
9486881,38991,2020-11-27 21:38:38,model_2,04.33.1261,5,NaN
9486882,38991,2020-11-28 00:13:46,model_2,04.33.1261,26,1


,user_id,time,model_nm,fwver,errtype,errcode
10425470,39894,2020-11-28 14:46:05,model_1,04.16.3553,20,1
10425471,39894,2020-11-28 14:46:21,model_1,04.16.3553,26,1
10425472,39894,2020-11-28 14:47:12,model_1,04.16.3553,5,-1010
10425473,39894,2020-11-28 14:47:12,model_1,04.16.3553,5,NaN
10425474,39894,2020-11-28 14:47:57,model_1,04.16.3553,32,80


In [58]:
# 결측치 채우기
test_err.iloc[937967, 5] = '40053'
test_err.iloc[4038892, 5] = '40053'
test_err.iloc[9486881, 5] = '40053'
test_err.iloc[10425473, 5] = '-1010'

In [59]:
test_err.isnull().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     0
dtype: int64

### test_quality

In [60]:
test_quality.isnull().sum()

time               0
user_id            0
fwver          22764
quality_0     106584
quality_1         11
quality_2      21115
quality_3          0
quality_4          0
quality_5         44
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

In [61]:
# fwver결측치 제거

fwver_null_index = list(test_quality[test_quality.fwver.isnull()].index)

test_quality = test_quality.drop(fwver_null_index, axis=0)
test_quality.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 725208 entries, 0 to 747971
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        725208 non-null  datetime64[ns]
 1   user_id     725208 non-null  int64         
 2   fwver       725208 non-null  object        
 3   quality_0   639780 non-null  float64       
 4   quality_1   725197 non-null  object        
 5   quality_2   725177 non-null  float64       
 6   quality_3   725208 non-null  int64         
 7   quality_4   725208 non-null  int64         
 8   quality_5   725164 non-null  object        
 9   quality_6   725208 non-null  int64         
 10  quality_7   725208 non-null  object        
 11  quality_8   725208 non-null  object        
 12  quality_9   725208 non-null  object        
 13  quality_10  725208 non-null  object        
 14  quality_11  725208 non-null  int64         
 15  quality_12  725208 non-null  int64         
dtypes:

In [62]:
# quality_0 ~ 12 최빈값으로 채우기

for i in test_quality.columns[3:]:
    test_quality[i] = test_quality[i].fillna(test_quality[i].mode()[0])

In [63]:
test_quality.isnull().sum()

time          0
user_id       0
fwver         0
quality_0     0
quality_1     0
quality_2     0
quality_3     0
quality_4     0
quality_5     0
quality_6     0
quality_7     0
quality_8     0
quality_9     0
quality_10    0
quality_11    0
quality_12    0
dtype: int64

### train_problem

In [64]:
train_problem.isnull().sum()

user_id    0
time       0
dtype: int64

### 각 dataframe 중복값 제거

In [65]:
# train_err의 중복값 제거
train_err = train_err.drop_duplicates(subset=None, keep='first')
print(train_err.shape)

# train_quality 중복값 제거
train_quality = train_quality.drop_duplicates(subset=None, keep='first')
print(train_quality.shape)

# test_err 중복값 제거
test_err = test_err.drop_duplicates(subset=None, keep='first')
print(test_err.shape)

# test_quality 중복값 제거)
test_quality = test_quality.drop_duplicates(subset=None, keep='first')
print(test_quality.shape)

(15368001, 6)
(272452, 16)
(15527221, 6)
(237629, 16)


## encoding

### quality_0~12 String to Int

In [66]:
# train/test_quality에서 쉼표로 인해 string형이 된  data 변경

def strToInt(x):
    if type(x) == str:
        x = x.replace(",", "")
        x = int(x)
        return x
    else:
        x = int(x)
        return x

In [67]:
# train_quality
for i in train_quality.columns[3:]:
    train_quality[i] = train_quality[i].apply(lambda x: strToInt(x))
    
# test_quality
for i in test_quality.columns[3:]:
    test_quality[i] = test_quality[i].apply(lambda x: strToInt(x))

In [68]:
# 결측치 처리 완료된 dataframe 저장
new_path = './preprocessed data/'

train_err.to_csv(new_path+'new_train_err.csv', index=False)
train_quality.to_csv(new_path+'new_train_quality.csv', index=False)
train_problem.to_csv(new_path+'new_train_problem.csv', index=False)
test_err.to_csv(new_path+'new_test_err.csv', index=False)
test_quality.to_csv(new_path+'new_test_quality.csv', index=False)